In [1]:
import time
import toml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from order_side import OrderSide
from ema_strategy import EMA_strategy

from binance.client import Client
from binance import ThreadedWebsocketManager

plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams["figure.dpi"] = 200
plt.rcParams.update({'font.size': 18})

In [3]:
cfg = toml.load("../api/configuration.toml")
pkey = cfg["auth"]["pkey"]
skey = cfg["auth"]["skey"]

client = Client(pkey, skey)

In [4]:
class Trader:
    def __init__(self, async_optimization=False):
        self.base_arr = []
        self.quote_arr = []
        self.price_history = []
        self.base = 1
        self.quote = 0
        self.tr = EMA_strategy()
        self.optimization_period = 50
        self.optimize_async_result = None
        self.num_buys = 0
        self.num_sells = 0
        self.async_optimization = async_optimization
    
    def trade(self, price):
        self.price_history.append(price)
        if len(self.price_history) < self.optimization_period:
            return OrderSide.NO_OP
        elif len(self.price_history) == self.optimization_period:
            if self.async_optimization:
                self.optimize_async_result = self.tr.optimize_multiprocess_async(self.price_history[:])
            else:
                self.tr.optimize_multiprocess(self.price_history[:])
            return OrderSide.NO_OP
        elif self.async_optimization:
            if not self.optimize_async_result.ready():
                return OrderSide.NO_OP

        traded = False
        self.quote *= 1 + (self.price_history[-1] - self.price_history[-2]) / self.price_history[-2]

        action = self.tr.action(self.price_history[-1])
        if action == OrderSide.BUY:  # buy
            if self.base > 0: 
                self.quote += .999 * self.base
                self.base = 0
                self.num_buys += 1
                traded = True
        elif action == OrderSide.SELL:  # sell
            if self.quote > 0:
                self.base += 0.999 * self.quote
                self.quote = 0
                self.num_sells += 1
                traded = True

        self.base_arr.append(self.base)
        self.quote_arr.append(self.quote)

        if len(self.price_history) % self.optimization_period == 0:
            if self.async_optimization:
                self.tr.optimize_multiprocess_async(self.price_history[-self.optimization_period:-1])
            else:
                self.tr.optimize_multiprocess(self.price_history[-self.optimization_period:-1])
            print(f"base: {self.base}\tquote: {self.quote}\tNUM BUYS: {self.num_buys}\tNUM SELLS: {self.num_sells}")

        if traded:
            return action
        return OrderSide.NO_OP

In [5]:
t = Trader(async_optimization=True)

def cb(msg):
    t.trade(float(msg["c"]))

bm = ThreadedWebsocketManager(api_key=pkey, api_secret=skey)
bm.start_symbol_ticker_socket('MATICBUSD', cb)
bm.start()

KeyboardInterrupt: 

In [ ]:
t = Trader()

history = pd.read_csv('../historical_data/binance_DOGEBTC_1m.csv') # dogecoin data
brices = history['open'].tolist()

bases = []
quotes = []

for p in brices:
    t.trade(p)

SET PARAMS: 0.0 9 0.0 0
SET PARAMS: 0.2 5 0.2 0
base: 1	quote: 0.0	NUM BUYS: 0	NUM SELLS: 0
SET PARAMS: 0.2 5 0.2 0
base: 0	quote: 0.9990000000000001	NUM BUYS: 1	NUM SELLS: 0
SET PARAMS: 0.2 5 0.2 0
base: 0	quote: 0.9696176470588237	NUM BUYS: 1	NUM SELLS: 0
SET PARAMS: 0.0 9 0.0 0
base: 0	quote: 0.9696176470588237	NUM BUYS: 1	NUM SELLS: 0
SET PARAMS: 0.2 5 0.2 0
base: 0	quote: 0.9402352941176483	NUM BUYS: 1	NUM SELLS: 0
SET PARAMS: 0.0 9 0.0 0
base: 0	quote: 0.9402352941176483	NUM BUYS: 1	NUM SELLS: 0
SET PARAMS: 0.0 9 0.0 0
base: 0	quote: 0.9696176470588248	NUM BUYS: 1	NUM SELLS: 0
SET PARAMS: 0.2 5 0.2 0
base: 0	quote: 0.9696176470588248	NUM BUYS: 1	NUM SELLS: 0
SET PARAMS: 0.6 1 0.6 2
base: 0	quote: 0.999000000000002	NUM BUYS: 1	NUM SELLS: 0
SET PARAMS: 0.2 5 0.2 0
base: 0	quote: 0.9990000000000024	NUM BUYS: 1	NUM SELLS: 0
SET PARAMS: 0.0 9 0.0 0
base: 0	quote: 0.9970029990000026	NUM BUYS: 2	NUM SELLS: 1
SET PARAMS: 0.2 5 0.2 0
base: 0	quote: 1.0574274231818208	NUM BUYS: 2	NUM SELLS

In [ ]:
client.get_ticker("MATICBUSD")